# Lab 03 (Intermediate) — Python for AI

Python 기초를 이미 알고 있다는 전제 하에, 데이터 분석·머신러닝에서 실제로 자주 쓰이는 패턴을 집중적으로 학습합니다.

**학습 목표:**
1. Python 중급 패턴 — comprehension, OOP, 예외처리, 타입힌트
2. NumPy 심화 — 선형대수, 벡터화 연산 vs 루프 성능 비교
3. Pandas 심화 — 실제 데이터 클리닝, merge, pivot, apply

In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# plt.rcParams["font.family"] = "AppleGothic"
plt.rcParams["font.family"] = "NanumBarunGothic"
plt.rcParams["axes.unicode_minus"] = False
sns.set_theme(style="whitegrid")

---
## Part 1. Python 중급 패턴

### 1-1. Comprehension

In [90]:
# List comprehension — 가장 흔하게 사용
squares = [x**2 for x in range(10) if x % 2 == 0]
print("짝수 제곱:", squares)

# Dict comprehension
word_len = {w: len(w) for w in ["apple", "banana", "cherry"]}
print("단어 길이:", word_len)

# Set comprehension — 중복 제거
labels = ["cat", "dog", "cat", "bird", "dog"]
unique_labels = {lbl.upper() for lbl in labels}
print("고유 레이블:", unique_labels)

# 중첩 comprehension — 2D → 1D flatten
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
flat = [v for row in matrix for v in row]
print("flatten:", flat)

짝수 제곱: [0, 4, 16, 36, 64]
단어 길이: {'apple': 5, 'banana': 6, 'cherry': 6}
고유 레이블: {'BIRD', 'CAT', 'DOG'}
flatten: [1, 2, 3, 4, 5, 6, 7, 8, 9]


### 1-2. Lambda · map · filter · zip

In [91]:
# lambda — 이름 없는 짧은 함수
normalize = lambda x, lo, hi: (x - lo) / (hi - lo)
print(normalize(75, 0, 100))

# map — 각 요소에 함수 적용
raw = [" Alice ", " Bob ", " Charlie "]
cleaned = list(map(str.strip, raw))
print(cleaned)

# filter — 조건을 만족하는 요소만
scores = [55, 72, 88, 91, 43, 66]
passed = list(filter(lambda s: s >= 60, scores))
print("통과:", passed)

# zip — 두 시퀀스를 묶어 쌍으로
names  = ["Alice", "Bob", "Charlie"]
scores = [88, 75, 92]
paired = list(zip(names, scores))
print("쌍:", paired)

# zip으로 딕셔너리 생성
record = dict(zip(names, scores))
print("딕셔너리:", record)

0.75
['Alice', 'Bob', 'Charlie']
통과: [72, 88, 91, 66]
쌍: [('Alice', 88), ('Bob', 75), ('Charlie', 92)]
딕셔너리: {'Alice': 88, 'Bob': 75, 'Charlie': 92}


### 1-3. 예외 처리 (Exception Handling)

In [92]:
def safe_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        print("오류: 0으로 나눌 수 없습니다.")
        return None
    except TypeError as e:
        print(f"타입 오류: {e}")
        return None

print(safe_divide(10, 2))
print(safe_divide(10, 0))
print(safe_divide(10, "a"))

5.0
오류: 0으로 나눌 수 없습니다.
None
타입 오류: unsupported operand type(s) for /: 'int' and 'str'
None


In [93]:
# 데이터 파이프라인에서 예외 처리 활용
raw_data = ["3.14", "2.71", "N/A", "1.41", ""]

def to_float(s):
    return float(s)
    
cleaned = [to_float(v) for v in raw_data]
print(cleaned)

ValueError: could not convert string to float: 'N/A'

In [94]:
# 데이터 파이프라인에서 예외 처리 활용
raw_data = ["3.14", "2.71", "N/A", "1.41", ""]

def to_float(s):
    try:
        return float(s)
    except (ValueError, TypeError):
        return np.nan

cleaned = [to_float(v) for v in raw_data]
print(cleaned)

[3.14, 2.71, nan, 1.41, nan]


### 1-4. 클래스 (OOP 기초)

In [95]:
class MyStudent:
    name:   str
    score: float
    
    def __init__(self, name, score):
        self.name = name 
        self.score = score 

    
bob = MyStudent("Bob", 87.)

In [96]:
print(bob)

In [97]:
print(f'name: {bob.name}' )
print(f'score: {bob.score}')

name: Bob
score: 87.0


In [98]:
class MyStudent:
    name:   str
    score: float
    
    def __init__(self, name, score):
        self.name = name 
        self.score = score 
        
    def __repr__(self):
        return f"MyStudent(name={self.name!r}, score={self.score})"

    def __str__(self):
        return f"{self.name}: {self.score}"

    
bob = MyStudent("Bob", 87.)
print(bob)

Bob: 87.0


In [99]:
from dataclasses import dataclass, field
from typing import List

@dataclass
class Student:
    name:   str
    scores: List[float] = field(default_factory=list)

    @property
    def average(self) -> float:
        return sum(self.scores) / len(self.scores) if self.scores else 0.0

    @property
    def grade(self) -> str:
        avg = self.average
        if avg >= 90: return "A"
        if avg >= 80: return "B"
        if avg >= 70: return "C"
        return "F"

    def __repr__(self):
        return f"{self.name} | 평균 {self.average:.1f} | 학점 {self.grade}"


students = [
    Student("Alice",   [88, 92, 85]),
    Student("Bob",     [72, 68, 75]),
    Student("Charlie", [95, 98, 91]),
]

print(f'Alice: average score is {students[0].average}.')

for s in students:
    print(s)

# 정렬
top = sorted(students, key=lambda s: s.average, reverse=True)
print("\n순위:", [s.name for s in top])

Alice: average score is 88.33333333333333.
Alice | 평균 88.3 | 학점 B
Bob | 평균 71.7 | 학점 C
Charlie | 평균 94.7 | 학점 A

순위: ['Charlie', 'Alice', 'Bob']


### 1-5. 타입 힌트 (Type Hints)

In [100]:
from typing import Optional, Tuple

def train_test_split_idx(
    n: int,
    test_ratio: float = 0.2,
    seed: Optional[int] = None
) -> Tuple[List[int], List[int]]:
    """인덱스를 train/test 로 분할."""
    rng = np.random.default_rng(seed)
    idx = rng.permutation(n).tolist()
    split = int(n * (1 - test_ratio))
    return idx[:split], idx[split:]

train_idx, test_idx = train_test_split_idx(20, seed=42)
print("train:", train_idx)
print("test: ", test_idx)

train: [15, 9, 14, 7, 12, 10, 6, 19, 3, 0, 16, 5, 11, 18, 2, 4]
test:  [17, 1, 13, 8]


> **Exercise 1.** 아래 함수를 완성하세요.  
> `normalize_list(data: List[float]) -> List[float]`  
> — 리스트를 Min-Max 정규화([0, 1] 범위)하여 반환합니다. 예외 처리(길이 0, 모두 같은 값)도 포함하세요.

In [101]:
def normalize_list(data: List[float]) -> List[float]:
    # Your code here
    pass

print(normalize_list([10, 20, 30, 40, 50]))  # [0.0, 0.25, 0.5, 0.75, 1.0]
print(normalize_list([5, 5, 5]))             # 모두 같은 값 → 처리 필요

None
None


### 1-6. 파일 입출력 (File I/O)

In [102]:
# 저장
with open("training_log.txt", "w", encoding="utf-8") as f:
    f.write("Epoch 1: Loss 0.50, Accuracy 80%\n") 
    f.write("Epoch 2: Loss 0.35, Accuracy 88%\n")
    f.write("Epoch 3: Loss 0.10, Accuracy 95%\n")


In [103]:
# 읽기
with open("training_log.txt", "r", encoding="utf-8") as f:
    for line in f:
        clean_line = line.strip() 
        print(f"읽은 데이터: {clean_line}")

읽은 데이터: Epoch 1: Loss 0.50, Accuracy 80%
읽은 데이터: Epoch 2: Loss 0.35, Accuracy 88%
읽은 데이터: Epoch 3: Loss 0.10, Accuracy 95%


---
## Part 2. NumPy 중급

### 2-1. 브로드캐스팅 규칙

In [104]:
# 규칙: 차원이 맞지 않으면 크기가 1인 차원을 확장
A = np.array([[1], [2], [3]])   # shape (3, 1)
B = np.array([10, 20, 30])      # shape (3,) → (1, 3)

print("A shape:", A.shape)
print("B shape:", B.shape)
print("A + B:\n", A + B)        # 결과 shape (3, 3), A는 열복사, B는 행복사

A shape: (3, 1)
B shape: (3,)
A + B:
 [[11 21 31]
 [12 22 32]
 [13 23 33]]


In [105]:
# 실용 예 — 데이터 표준화 (Z-score)
X = np.array([[2, 4, 6],
              [1, 3, 5],
              [3, 5, 7]], dtype=float)
print(X.shape)
mean = X.mean(axis=0)  # 열(feature)별 평균, shape (3,) [2. 4. 6.] -> (1,3)-> (3,3)
std  = X.std(axis=0)   # 열별 표준편차 [0.81649658 0.81649658 0.81649658] (3,) -> (1,3) -> (3,3)

Z = (X - mean) / std   # 브로드캐스팅으로 한 번에 처리
print("표준화:\n", Z.round(3))
print("열별 평균:", Z.mean(axis=0).round(10))  # ≈ 0
print("열별 표준편차:", Z.std(axis=0).round(10))   # ≈ 1

(3, 3)
표준화:
 [[ 0.     0.     0.   ]
 [-1.225 -1.225 -1.225]
 [ 1.225  1.225  1.225]]
열별 평균: [0. 0. 0.]
열별 표준편차: [1. 1. 1.]


### 2-2. 벡터화 vs 루프 — 성능 비교

In [106]:
import time

N = 1_000_000
data = np.random.rand(N)

# 방법 1: Python 루프
start = time.perf_counter()
total = 0
for v in data:
    total += v
t_loop = time.perf_counter() - start

# 방법 2: NumPy 벡터화
start = time.perf_counter()
total_np = data.sum()
t_np = time.perf_counter() - start

print(f"Python 루프 : {t_loop*1000:.1f} ms")
print(f"NumPy 벡터화: {t_np*1000:.3f} ms")
print(f"속도 향상   : {t_loop / t_np:.0f}x")

Python 루프 : 83.4 ms
NumPy 벡터화: 0.232 ms
속도 향상   : 359x


### 2-3. 선형대수 (np.linalg)

In [107]:
# 행렬 곱
A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8]])
print("A @ B:\n", A @ B)              # @ 연산자 = matmul

# 역행렬
A_inv = np.linalg.inv(A)
print("\nA 역행렬:\n", A_inv.round(3))
print("A @ A_inv (≈ I):\n", (A @ A_inv).round(10))

# 고유값 분해 (PCA 등에서 핵심)
eigenvalues, eigenvectors = np.linalg.eig(A)
print("\n고유값:", eigenvalues.round(3))
print("고유벡터:\n", eigenvectors.round(3))

A @ B:
 [[19 22]
 [43 50]]

A 역행렬:
 [[-2.   1. ]
 [ 1.5 -0.5]]
A @ A_inv (≈ I):
 [[1. 0.]
 [0. 1.]]

고유값: [-0.372  5.372]
고유벡터:
 [[-0.825 -0.416]
 [ 0.566 -0.909]]


In [108]:
# 최소제곱법으로 선형 회귀 계수 구하기
# y = w0 + w1*x  →  X @ w = y
rng = np.random.default_rng(0)
x = np.linspace(0, 10, 50)
y = 2.5 * x + 1.0 + rng.normal(0, 1, 50)  # 실제 w0=1, w1=2.5

X = np.column_stack([np.ones(len(x)), x])  # 편향 항 추가
w, residuals, rank, sv = np.linalg.lstsq(X, y, rcond=None)

print(f"추정 w0 (절편): {w[0]:.3f}  (실제: 1.0)")
print(f"추정 w1 (기울기): {w[1]:.3f}  (실제: 2.5)")

추정 w0 (절편): 0.549  (실제: 1.0)
추정 w1 (기울기): 2.616  (실제: 2.5)


### 2-4. 유용한 함수들

In [109]:
a = np.array([3, 1, 4, 1, 5, 9, 2, 6, 5, 3])

# np.where — 조건에 따라 값 선택
clipped = np.where(a > 5, 5, a)   # 5 초과면 5로 대체
print("클리핑:", clipped)

# np.unique — 고유값과 빈도
vals, counts = np.unique(a, return_counts=True)
print("고유값:", vals)
print("빈도:  ", counts)

# np.argsort — 정렬된 인덱스
idx = np.argsort(a)[::-1]   # 내림차순
print("내림차순 인덱스:", idx)
print("정렬 결과:      ", a[idx])
idx = np.argsort(a)
print("오름차순 정렬 결과:      ", a[idx])

클리핑: [3 1 4 1 5 5 2 5 5 3]
고유값: [1 2 3 4 5 6 9]
빈도:   [2 1 2 1 2 1 1]
내림차순 인덱스: [5 7 8 4 2 9 0 6 3 1]
정렬 결과:       [9 6 5 5 4 3 3 2 1 1]
오름차순 정렬 결과:       [1 1 2 3 3 4 5 5 6 9]


> **Exercise 2.** `(100, 5)` 크기의 랜덤 행렬 `X`를 생성하고,  
> 각 열(feature)을 Z-score 정규화한 뒤 정규화 후 각 열의 평균과 표준편차를 확인하세요.

In [110]:
# Your code here


---
## Part 3. Pandas 중급

### 3-1. Titanic 데이터 로딩

In [111]:
titanic = sns.load_dataset("titanic")
print(titanic.shape)
titanic.head()

(891, 15)


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


### 3-2. 데이터 클리닝 파이프라인

In [112]:
# 결측치 현황
missing = titanic.isnull().sum()
missing_pct = (missing / len(titanic) * 100).round(1)
pd.DataFrame({"결측 수": missing, "결측 비율(%)": missing_pct}).query("`결측 수` > 0")

,결측 수,결측 비율(%)
age,177,19.9
embarked,2,0.2
deck,688,77.2
embark_town,2,0.2


In [113]:
df = titanic.copy()

# age: 중앙값으로 대체
df["age"] = df["age"].fillna(df["age"].median())

# embarked: 최빈값으로 대체
df["embarked"] = df["embarked"].fillna(df["embarked"].mode()[0])

# deck: 결측 비율이 너무 높아 열 제거
df = df.drop(columns=["deck"])

# 중복 행 확인 및 제거
print("중복 행:", df.duplicated().sum())

print("\n클리닝 후 결측치:")
print(df.isnull().sum()[df.isnull().sum() > 0])

중복 행: 116

클리닝 후 결측치:
embark_town    2
dtype: int64


### 3-3. 특성 엔지니어링

In [114]:
# 가족 크기
df["family_size"] = df["sibsp"] + df["parch"] + 1
df["is_alone"] = (df["family_size"] == 1).astype(int)

# 나이 그룹
df["age_group"] = pd.cut(
    df["age"],
    bins=[0, 12, 18, 60, 100],
    labels=["child", "teen", "adult", "senior"]
)

# 운임 로그 변환 (왜도 감소)
df["log_fare"] = np.log1p(df["fare"])

df[["family_size", "is_alone", "age_group", "log_fare"]].head()

,family_size,is_alone,age_group,log_fare
0,2,0,adult,2.110213
1,2,0,adult,4.280593
2,1,1,adult,2.188856
3,2,0,adult,3.990834
4,1,1,adult,2.202765


### 3-4. groupby 심화

In [115]:
# 다중 집계
survival_stats = df.groupby(["pclass", "sex"])["survived"].agg(
    count="count",
    survivors="sum",
    survival_rate="mean"
).round(3)

survival_stats

count  survivors  survival_rate
pclass sex                                    
1      female     94         91          0.968
       male      122         45          0.369
2      female     76         70          0.921
       male      108         17          0.157
3      female    144         72          0.500
       male      347         47          0.135

In [116]:
# transform — 그룹 통계를 원본 행에 붙이기
df["class_avg_fare"] = df.groupby("pclass")["fare"].transform("mean")
df["fare_vs_class_avg"] = df["fare"] - df["class_avg_fare"]

df[["pclass", "fare", "class_avg_fare", "fare_vs_class_avg"]].head(8)

,pclass,fare,class_avg_fare,fare_vs_class_avg
0,3,7.2500,13.675550,-6.425550
1,1,71.2833,84.154687,-12.871387
2,3,7.9250,13.675550,-5.750550
3,1,53.1000,84.154687,-31.054687
4,3,8.0500,13.675550,-5.625550
5,3,8.4583,13.675550,-5.217250
6,1,51.8625,84.154687,-32.292187
7,3,21.0750,13.675550,7.399450


### 3-5. pivot_table

In [117]:
pivot = pd.pivot_table(
    df,
    values="survived",
    index="pclass",
    columns="sex",
    aggfunc="mean"
).round(3)

pivot

sex,female,male
pclass,,
1,0.968,0.369
2,0.921,0.157
3,0.500,0.135


### 3-6. apply + 커스텀 함수

In [118]:
def fare_tier(row):
    """객실 등급과 운임을 기반으로 티어 분류"""
    if row["pclass"] == 1 and row["fare"] > 50:
        return "premium"
    elif row["pclass"] <= 2:
        return "standard"
    else:
        return "economy"

df["tier"] = df.apply(fare_tier, axis=1)
df[["pclass", "fare", "tier", "survived"]].head(10)

,pclass,fare,tier,survived
0,3,7.2500,economy,0
1,1,71.2833,premium,1
2,3,7.9250,economy,1
3,1,53.1000,premium,1
4,3,8.0500,economy,0
5,3,8.4583,economy,0
6,1,51.8625,premium,0
7,3,21.0750,economy,0
8,3,11.1333,economy,1
9,2,30.0708,standard,1


> **Exercise 3.** 나이 그룹(`age_group`)별 생존율을 계산하고,  
> 어느 그룹의 생존율이 가장 높은지 확인하세요.

In [119]:
# Your code here


---
## Summary

| 파트 | 핵심 개념 |
|---|---|
| Python 중급 | comprehension, lambda/map/filter/zip, 예외처리, dataclass, 타입힌트 |
| NumPy 심화 | 브로드캐스팅, 벡터화 성능, 선형대수(linalg), 최소제곱법 |
| Pandas 심화 | 데이터 클리닝, 특성 엔지니어링, groupby+transform, pivot_table, apply |
